
# *Advanced Data Analysis and Modelling*

## *7.5.4 Example of maximizing Flow in a Transportation Network*

Authors: Tim Diller and Gregor Henze

Created: March 20, 2024

7.5.4 Example of Maximizing Flow
in a Transportation Network
This example is taken from Vugrin et al. (2014) to illustrate
how to analyze transportation networks. Figure 7.20
represents a simple network with 7 nodes and 12 links
where the objective is to maximize flow from one starting
node 1 to another specified end node 7. The limiting
capacities of various links are specified (and indicated in
the figure). A fictitious return link (shown dotted) with infi-
nite capacity needs to be introduced to complete the circuit.
The optimization model for this flow problem can be
framed as:

*Maximise the flow in edge 7_1 without exceeding any of the flow limits of any of the edges, and without adding any sources or sinks to the network.*


For the uninterrupted case, i.e., when no links are broken,
the maximum flow is 14 units. The same
equations can be modified to analyze the situation when
one or more of the links breaks. Such analyses can be
performed assuming different scenarios of one or more link
breakages. Such types of evaluations are usually done in the
framework of reliability analyses during the design of the
networks

To simulate and visualize such a problem in Python, we need to first import the relevant packages.
Note that pulp is not standard in Colab, so we need to download both pulp and the open source glpk solver.

In [ ]:
# import required packages
!pip install pulp
!apt-get install -y -qq glpk-utils
import pulp as pl
import numpy as np
import plotly.graph_objects as go

The first step in pulp is to create a linear problem object.

In [ ]:
# defining the network
network_model = pl.LpProblem("shift_model", pl.LpMaximize)

Then we create the individual variables (in this case the edges), and adjust their capacity constraints.

In [ ]:
# defining the variables
edge_12 = pl.LpVariable("edge_12", cat="Continuous", lowBound=0,upBound=5)
edge_13 = pl.LpVariable("edge_13", cat="Continuous", lowBound=0,upBound=7)
edge_14 = pl.LpVariable("edge_14", cat="Continuous", lowBound=0,upBound=4)
edge_23 = pl.LpVariable("edge_23", cat="Continuous", lowBound=0,upBound=1)
edge_34 = pl.LpVariable("edge_34", cat="Continuous", lowBound=0,upBound=2)
edge_25 = pl.LpVariable("edge_25", cat="Continuous", lowBound=0,upBound=3)
edge_35 = pl.LpVariable("edge_35", cat="Continuous", lowBound=0,upBound=4)
edge_36 = pl.LpVariable("edge_36", cat="Continuous", lowBound=0,upBound=5)
edge_46 = pl.LpVariable("edge_46", cat="Continuous", lowBound=0,upBound=4)
edge_65 = pl.LpVariable("edge_65", cat="Continuous", lowBound=0,upBound=1)
edge_57 = pl.LpVariable("edge_57", cat="Continuous", lowBound=0,upBound=9)
edge_67 = pl.LpVariable("edge_67", cat="Continuous", lowBound=0,upBound=6)
edge_71 = pl.LpVariable("edge_71", cat="Continuous", lowBound=0)  # no upper bound, we want to maximise edge71

# package into a list of easier reference
listofedges = [edge_12, edge_13, edge_14, edge_23, edge_34, edge_25, edge_35, edge_36, edge_46, edge_65, edge_57, edge_67]


Then we specify the constraints. Kirchhoffs law needs to apply to all nodes, so no node can have a source or a sink.


In [ ]:
# kirchhoffs law around all nodes of the network
network_model += edge_71 - edge_12 -edge_13 - edge_14 == 0  # node 1
network_model += edge_12 - edge_23 - edge_25 == 0  # node 2
network_model += edge_13 + edge_23 - edge_35 - edge_36 - edge_34 == 0  # node 3
network_model += edge_14 + edge_34 - edge_46 == 0  # node 4
network_model += edge_25 + edge_35 + edge_65 - edge_57  == 0  # node 5
network_model += edge_46 + edge_36 - edge_65 - edge_67 == 0  # node 6
network_model += edge_57 + edge_67 - edge_71 == 0  # node 7

Finally, the objective function is to maximise the flow through the edge 7_1.

This is an optimization trick: In generating an additional imaginary 'back-flow edge', we can avoid adding any sinks or sources to the network, which would then need the additional constraint of being of equal size.

In [ ]:
# add the objective function
network_model += edge_71

Now we can solve the problem, and print the values of the individual variables

In [ ]:
# solve the model, and print the maximum flow
network_model.solve(pl.GLPK(options=[]))
print("max flow: ", edge_71.value())



max flow:  14.0


In [ ]:
network_model += edge_13 == 0

Now we can define a function that visualizes the network, and the flow that it has based on our optimization

In [ ]:
def visualize_network(listofedges):

    def plot_line(fig, start_index, end_index, width, color, name):
        start_point = points[start_index]
        end_point = points[end_index]

        x_start, y_start = start_point
        x_end, y_end = end_point

        fig.add_trace(go.Scatter(
            x=[x_start, x_end],
            y=[y_start, y_end],
            mode='lines',
            line=dict(color=color, width=width),  # Set line color and width
            name=name,  # Set name for legend
        ))



    points = [[0, 5],
                [3, 8],
                [4, 5],
                [5, 2],
                [8, 7],
                [7, 3],
                [11, 5]]

    edges = [[1, 2],
            [1, 3],
            [1, 4],
            [2, 3],
            [3, 4],
            [2, 5],
            [3, 5],
            [3, 6],
            [4, 6],
            [6, 5],
            [5, 7],
            [6, 7],
            [7, 1],
    ]

    capacities = [5, 7, 4, 1, 2, 3, 4, 5, 4, 1, 9, 6]

    x_coords = [point[0] for point in points]
    y_coords = [point[1] for point in points]

    # Create a scatter plot with points
    # fig = go.Figure(data=go.Scatter(x=x_coords, y=y_coords, mode='markers', marker=dict(size=10, color='darkblue', line=dict(color='black', width=1), opacity=0.7)))
    fig = go.Figure()

    for edge, capacity in zip(edges, capacities):
        plot_line(fig, edge[0] - 1, edge[1] - 1, capacity * 3, "grey", "pipe_" + str(edge[0]) + str(edge[1]))

    for edge, flow in zip(edges, listofedges):
        plot_line(fig, edge[0] - 1, edge[1] - 1, flow.value() * 3, "red", "flow_" + str(edge[0]) + str(edge[1]))

    for i, point in enumerate(points):
        x, y = point
        fig.add_trace(go.Scatter(
            x=[x],
            y=[y],
            mode='markers',
            marker=dict(size=50, color='darkblue', line=dict(color='black', width=1), opacity=0.9),  # Adjust marker size, border color, and opacity
            name=f'Node {i+1}',  # Set name for legend
            text=[f'{i+1}'],  # Text for annotation
            textposition="middle center",  # Position of annotation
        ))


    fig.update_layout(
        width=8.27*100,  # Convert A4 width from inches to pixels (1 inch = 100 pixels)
        height=8.27*100/1.41,  # Set height to maintain A4 aspect ratio
        xaxis=dict(showgrid=False),  # Remove grid from x-axis
        yaxis=dict(showgrid=False),  # Remove grid from y-axis
        plot_bgcolor='rgba(0,0,0,0)'  # Make plot background transparent
    )

    fig.show()



We visualize the solution, where the capacity of each node is shown in grey, and the actual flow is shown in red.

In [ ]:
visualize_network(listofedges)

Now we can simulate one of the edges failing (by setting its flow to zero) and see how the network reacts

In [ ]:
network_model += edge_13 == 0  # replace with any edge or list of edges you want to take out

In [ ]:
# solve the model, and print the maximum flow
network_model.solve(pl.GLPK(options=[]))
print("max flow: ", edge_71.value())
visualize_network(listofedges)

max flow:  8.0


#### Note about PuLP ####

Please note that PuLP does not support the deletion of constraints once they are added. So, in order to examine the effect of different individual edges breaking, the model needs to be reinitialized.

This can be done by rerunning the blocks [creating the problem](https://colab.research.google.com/drive/1w_xFhBpK1HeYenBcykgXuOELTVCzxMjH#scrollTo=vlunOJy_FBpi&line=1&uniqifier=1), the [variables](https://colab.research.google.com/drive/1w_xFhBpK1HeYenBcykgXuOELTVCzxMjH#scrollTo=DhOsArWbFGO6&line=4&uniqifier=1), [the constraints](https://colab.research.google.com/drive/1w_xFhBpK1HeYenBcykgXuOELTVCzxMjH#scrollTo=32hClhMwFP-B&line=3&uniqifier=1), and the [objective function](https://colab.research.google.com/drive/1w_xFhBpK1HeYenBcykgXuOELTVCzxMjH#scrollTo=Y5Pw_H39FbCh&line=2&uniqifier=1), and then adding the [individual constraint](https://colab.research.google.com/drive/1w_xFhBpK1HeYenBcykgXuOELTVCzxMjH#scrollTo=iEH0go_eWbDp&line=1&uniqifier=1) that should be updated.


## Conclusion ##

This notebook shows how to create a simple LP optimization problem within the python pulp package, and solve it using the GLPK solver.

It also visualizes the solution, and shows how failure of individual edges affects the overall performance of the network.


